<a href="https://colab.research.google.com/github/blankazselyke/Onlab/blob/DPO_alap/%C3%96nlab_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers
!pip3 install datasets
!pip3 install trl
!pip install -q -U bitsandbytes
!pip install accelerate
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments, AutoModelForQuestionAnswering
from trl import DPOTrainer, SFTTrainer
from transformers import BitsAndBytesConfig
from torch import bfloat16
import torch
from peft import LoraConfig,PeftModel, PeftConfig, get_peft_model, AutoPeftModelForCausalLM
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)
#Even though the weights are loaded in a 4-bit format,
#computations during forward and backward passes will be performed using bfloat16 data type,
#which is a 16-bit floating-point representation.
#bfloat16 strikes a balance between memory efficiency and computational precision.
model_name ="facebook/opt-350m"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
base_model.config.use_cache = False

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
base_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (

In [ ]:
proba = '/content/train.csv'

In [ ]:
def formatting_prompts_func(example):
  output_texts = []
  assert len(example["text"]) == len(example["label"])
  for i in range(len(example['text'])):
    text = f"### Input: ```{example['text'][i]}```\n ### Output: {example['label'][i]}"
    output_texts.append(text)
  return output_texts

In [ ]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [ ]:
sft_dataset = load_dataset("gbharti/finance-alpaca", split="train")
sft_datset_new = sft_dataset.select(range(10000))
sft_splitted_data = sft_datset_new.train_test_split(test_size=0.3,seed=42, shuffle=True)

sft_splitted_data["train"] = sft_splitted_data["train"].remove_columns(["input", "text"])
sft_splitted_data["test"] = sft_splitted_data["test"].remove_columns(["input", "text"])

sft_splitted_data["train"] = sft_splitted_data["train"].rename_column("output", "label")
sft_splitted_data["train"] = sft_splitted_data["train"].rename_column("instruction", "text")

sft_splitted_data["test"]  = sft_splitted_data["test"].rename_column("output", "label")
sft_splitted_data["test"]  = sft_splitted_data["test"].rename_column("instruction", "text")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
sft_splitted_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
# training the supervised model
"""
 max_step = epoch*(total_example/batch_size)
 logging_step = total_example/batch_size
 loggers = logging_step/epoch
"""
output_path = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/"
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["k_proj","v_proj","q_proj","out_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
MAX_STEPS =int(2 * len(sft_splitted_data["train"]) / 4)
args = TrainingArguments(
        output_path,
        overwrite_output_dir=True, # This reduces the amt of disk space that gets used.
        fp16=True,  # fp16 training to allow larger batch sizes to be used
        evaluation_strategy = "steps",
        save_strategy = "steps",
        learning_rate=5e-5,
        warmup_steps=500,
        warmup_ratio=0.1,
        do_train=True,
        do_eval=True,
        logging_steps=500,
        eval_steps=500,
        save_total_limit=1,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        max_steps=MAX_STEPS,
        weight_decay=0.01,
        dataloader_num_workers=4,
        load_best_model_at_end=True
)

trainer = SFTTrainer(
    model=base_model,
    train_dataset=sft_splitted_data["train"],
    eval_dataset=sft_splitted_data["test"],
    tokenizer=tokenizer,
    peft_config=config,
    formatting_func=formatting_prompts_func,
    dataset_text_field="text",
    args=args,         # Trainer arguments
    max_seq_length=256,
)
trainer.train()

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,3.943100,3.631700
1000,3.642500,3.555042
1500,3.548500,3.507818
2000,3.531900,3.470770
2500,3.472400,3.447563
3000,3.433500,3.430830
3500,3.431400,3.424745


TrainOutput(global_step=3500, training_loss=3.571900913783482, metrics={'train_runtime': 2171.7922, 'train_samples_per_second': 12.893, 'train_steps_per_second': 1.612, 'total_flos': 1059394415493120.0, 'train_loss': 3.571900913783482, 'epoch': 4.0})

In [ ]:
output_path = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/"
adapter_path = os.path.join(output_path,"checkpoint-3500")

In [ ]:
math_dataset = load_dataset("argilla/distilabel-math-preference-dpo", split="train")
def process_dataset(sample_data):
  return {
      "prompt": [f"Question: " + question + "\n\nAnswer: "
      for question in sample_data["instruction"]
      ],
      "chosen": sample_data["chosen_response"],
      "rejected": sample_data["rejected_response"]
  }

original_cols = math_dataset.column_names
math_dataset = math_dataset.map(process_dataset,batched=True,remove_columns=original_cols)
# let's divide the dataset
math_dataset_splits = math_dataset.train_test_split(test_size=0.2,seed=42,shuffle=True)
math_dataset_splits

Generating train split:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1934
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 484
    })
})

In [ ]:
# now we can train the DPO model based on the dataset we create
model = AutoPeftModelForCausalLM.from_pretrained(
 adapter_path,
 low_cpu_mem_usage=True,
 torch_dtype=torch.bfloat16,
 load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(adapter_path,trust_remote_code=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
MAX_STEPS =int(5 * len(math_dataset_splits["train"]) / 16)

output_path_dpo = os.path.join(output_path, "dpo_models")
args = TrainingArguments(
        output_path_dpo,
        overwrite_output_dir=True, # This reduces the amt of disk space that gets used.
        fp16=True,  # fp16 training to allow larger batch sizes to be used
        evaluation_strategy = "steps",
        save_strategy = "steps",
        learning_rate=5e-5,
        warmup_steps=500,
        warmup_ratio=0.1,
        do_train=True,
        do_eval=True,
        logging_steps=500,
        eval_steps=500,
        save_total_limit=1,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        max_steps=MAX_STEPS,
        weight_decay=0.01,
        dataloader_num_workers=4,
        load_best_model_at_end=True
)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["k_proj","v_proj","q_proj","out_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

dpo_trainer = DPOTrainer(
    model,
    args=args,
    beta=0.1,
    train_dataset=math_dataset_splits["train"],
    eval_dataset=math_dataset_splits["test"],
    tokenizer=tokenizer,
    peft_config=config,
    max_length=512,
)
dpo_trainer.train()

Map:   0%|          | 0/1934 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
500,0.849100,0.718492,0.442011,0.426724,0.506198,0.015287,-596.790955,-617.549927,-0.873143,-1.195648


TrainOutput(global_step=604, training_loss=0.8336948243198016, metrics={'train_runtime': 900.8222, 'train_samples_per_second': 2.682, 'train_steps_per_second': 0.67, 'total_flos': 0.0, 'train_loss': 0.8336948243198016, 'epoch': 1.25})

In [ ]:
!zip -r /content/lib.zip /content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500

  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/ (stored 0%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/README.md (deflated 66%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/adapter_model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/adapter_config.json (deflated 52%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/tokenizer_config.json (deflated 63%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/special_tokens_map.json (deflated 74%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/vocab.json (deflated 59%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint-3500/merges.txt (deflated 53%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/checkpoint

In [ ]:
!rm -rf /content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/dpo_models/checkpoint-11500

In [ ]:
adapter_model = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft/models/dpo_models/checkpoint-500/"
# tokenizer = AutoTokenizer.from_pretrained(adapter_model)
# model = AutoModelForCausalLM.from_pretrained(base_model)
# model.resize_token_embeddings(len(tokenizer))
# peftmodel = PeftModel.from_pretrained(model, adapter_model)

model = AutoPeftModelForCausalLM.from_pretrained(adapter_model, torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(adapter_model)

In [ ]:
model.eval()
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

</s>Question: Explain the concept of conditional probability and provide an example..
 Take a deep breath, think step by step, and give an accurate response

Answer: 

I'm not sure what you mean. I don't know how to explain it in any way other than "it's just a theory".
The idea is that if you have a set of values for a variable, then you can use the conditional probabilities (or the probability distribution) as a basis on which to calculate the value of each variable.
For example, suppose you want to find the value of a variable, but only one of those variables has a positive or negative probability.
****************************************************************************************************
</s>Question: Detail the steps involved in finding the solution to a linear equation with three variables..
 Take a deep breath, think step by step, and give an accurate response

Answer: _______________________________________________

1.
Step 1 -
The first step is to find the solution of